<a href="https://colab.research.google.com/github/soaring0616/some_stuff/blob/main/Ollama_%E9%81%A0%E7%AB%AF%E5%9F%B7%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#教學說明
本地主只是讓學生學會如何使用自行部署的 ollama 在雲端以及本地端都可以體驗大型語言模型的魅力

# 前置作業

Download and install ollama to the system


In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100  8575    0  8575    0     0  38994      0 --:--:-- --:--:-- --:--:-- 39155
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#@title 1. 保持此標籤處於活動狀態，以防止 Colab 斷開您的連接。 { display-mode: "form" }

#@markdown 在下方出現的音樂播放器上按播放鍵:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

使用 ngrok 提供 ollama 一個公開 IP 位置


In [ ]:
!pip install -UUU aiohttp pyngrok

In [ ]:
#@title 2. 設定遠端 ollama 模型

#@markdown 如果不確定 branch 名稱，請填寫「main」或留空。


import os
import asyncio

# 使用 #param 定義參數
authtoken = '' #@param {type:"string"}
ollama_run_command = 'phi' #@param ["zephyr", "phi", "mistral"]

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
    print('>>> starting', *cmd)
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

# 使用定義的參數
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', authtoken])
)

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'run', ollama_run_command]),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

>>> starting ngrok config add-authtoken 2d7e2R***LrhD_6zW5fS29dT3a2hpj28fYU
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ollama serve
>>> starting ollama run phi
>>> starting ngrok http --log stderr 11434
time=2024-03-02T07:12:03.850Z level=INFO source=images.go:710 msg="total blobs: 6"
time=2024-03-02T07:12:03.851Z level=INFO source=images.go:717 msg="total unused blobs removed: 0"
time=2024-03-02T07:12:03.851Z level=INFO source=routes.go:1019 msg="Listening on 127.0.0.1:11434 (version 0.1.27)"
time=2024-03-02T07:12:03.852Z level=INFO source=payload_common.go:107 msg="Extracting dynamic libraries..."
t=2024-03-02T07:12:04+0000 lvl=info msg="no configuration paths supplied"
t=2024-03-02T07:12:04+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2024-03-02T07:12:04+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2024-03-02T07:12:04+0000 lvl=info msg="starting web serv

## 本機端測試指令
```
curl https://3087-34-16-151-39.ngrok-free.app/api/generate -d '{
  "model": "phi",
  "prompt":"Why is the sky blue?",
}'
```

複製上述文字內容，於底下，並且修改兩處

1. 打上 `%%bash`
2. 複製打開的超連結 https://xxxxxxx.app 取代掉原本網址內容

In [ ]:
%%bash
curl https://3087-34-16-151-39.ngrok-free.app/api/generate -d '{
  "model": "phi",
  "prompt":"Why is the sky blue?",
}'